# Análisis de Regresión: Motor Trend Car Road Tests Arturo Ayala Hernández
Importamos las librerías necesarias y cargamos el dataset, eliminando la columna `model` ya que es un identificador de texto y no aporta valor predictivo numérico a la regresión.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge

# Cargar el archivo de datos (asegúrate de que el nombre coincida con tu archivo local)
df = pd.read_excel('Motor Trend Car Road Tests.xlsx')

# Eliminamos la columna 'model'
df_base = df.drop(columns=['model'])

## Actividad 1.1: Regresión con 'mpg' como salida (Variables continuas/ordinales)
Calcularemos el R2 del modelo completo, interpretaremos los coeficientes (betas), realizaremos un Train-Test Split (40% para entrenar) y probaremos regularización L2 (Ridge) ajustando el hiperparámetro lambda.

In [2]:
# Definir X e y
X_11 = df_base.drop(columns=['mpg'])
y_11 = df_base['mpg']

# 1. Modelo Completo
modelo_11 = LinearRegression()
modelo_11.fit(X_11, y_11)
print(f"R2 Modelo Completo: {modelo_11.score(X_11, y_11):.4f}")

print("\nBetas (Coeficientes):")
for feature, coef in zip(X_11.columns, modelo_11.coef_):
    print(f" - {feature}: {coef:.4f}")

# 2. Train-Test Split (40% train)
X_train_11, X_test_11, y_train_11, y_test_11 = train_test_split(X_11, y_11, train_size=0.4, random_state=42)
modelo_11_split = LinearRegression()
modelo_11_split.fit(X_train_11, y_train_11)
print(f"\nR2 Entrenamiento: {modelo_11_split.score(X_train_11, y_train_11):.4f}")
print(f"R2 Prueba: {modelo_11_split.score(X_test_11, y_test_11):.4f}")

# 3. Regularización L2 (Ridge)
print("\nRegularización Ridge con distintos Lambdas (Alphas):")
for alpha_val in [0.1, 1.0, 10.0, 100.0]:
    ridge_11 = Ridge(alpha=alpha_val)
    ridge_11.fit(X_train_11, y_train_11)
    r2_train = ridge_11.score(X_train_11, y_train_11)
    r2_test = ridge_11.score(X_test_11, y_test_11)
    print(f" Lambda = {alpha_val:5.1f} | Train R2: {r2_train:.4f} | Test R2: {r2_test:.4f}")

R2 Modelo Completo: 0.8690

Betas (Coeficientes):
 - cyl: -0.1114
 - disp: 0.0133
 - hp: -0.0215
 - drat: 0.7871
 - wt: -3.7153
 - qsec: 0.8210
 - vs: 0.3178
 - am: 2.5202
 - gear: 0.6554
 - carb: -0.1994

R2 Entrenamiento: 0.9982
R2 Prueba: -7.1071

Regularización Ridge con distintos Lambdas (Alphas):
 Lambda =   0.1 | Train R2: 0.9794 | Test R2: 0.2607
 Lambda =   1.0 | Train R2: 0.9279 | Test R2: 0.6311
 Lambda =  10.0 | Train R2: 0.8634 | Test R2: 0.6567
 Lambda = 100.0 | Train R2: 0.8073 | Test R2: 0.5990


**Interpretación 1.1:**
* **Betas Positivas (`drat`, `qsec`, `am`, `gear`):** Un aumento en estas características (ej. autos manuales o con más engranajes) favorece el rendimiento de combustible (mpg).
* **Betas Negativas (`wt`, `hp`, `cyl`):** Un auto más pesado, potente o con más cilindros reduce significativamente el `mpg`.
* **Desempeño y L2:** Al usar solo el 40% de los datos, el modelo sin regularizar se sobreajusta (R2 Test negativo). Al aplicar Ridge con un Lambda alrededor de 10.0, penalizamos los coeficientes, sacrificando exactitud en entrenamiento pero rescatando la capacidad predictiva en datos de prueba.

## Actividad 1.2: Regresión con 'qsec' como salida
Repetimos el proceso anterior, pero ahora intentamos predecir el tiempo en el cuarto de milla (`qsec`).

In [3]:
X_12 = df_base.drop(columns=['qsec'])
y_12 = df_base['qsec']

modelo_12 = LinearRegression()
modelo_12.fit(X_12, y_12)
print(f"R2 Modelo Completo: {modelo_12.score(X_12, y_12):.4f}")

print("\nBetas (Coeficientes):")
for feature, coef in zip(X_12.columns, modelo_12.coef_):
    print(f" - {feature}: {coef:.4f}")

X_train_12, X_test_12, y_train_12, y_test_12 = train_test_split(X_12, y_12, train_size=0.4, random_state=42)
modelo_12_split = LinearRegression()
modelo_12_split.fit(X_train_12, y_train_12)
print(f"\nR2 Entrenamiento: {modelo_12_split.score(X_train_12, y_train_12):.4f}")
print(f"R2 Prueba: {modelo_12_split.score(X_test_12, y_test_12):.4f}")

R2 Modelo Completo: 0.8747

Betas (Coeficientes):
 - mpg: 0.0690
 - cyl: -0.3627
 - disp: -0.0075
 - hp: -0.0016
 - drat: -0.1311
 - wt: 1.4963
 - vs: 0.9700
 - am: -0.9012
 - gear: -0.2013
 - carb: -0.2736

R2 Entrenamiento: 0.9989
R2 Prueba: -1.0013


**Interpretación 1.2:**
* **Betas:** El peso (`wt`) tiene un signo fuertemente positivo (hace el auto más lento), mientras que `hp` (caballos de fuerza) y transmisiones manuales (`am`) tienen signos negativos, reduciendo el tiempo del auto (haciéndolo más rápido).
* **Train/Test:** Nuevamente existe un sobreajuste evidente debido al alto número de variables frente a muy pocos datos de entrenamiento.

## Actividad 2.1: Regresión con 'mpg' (Usando Variables Dummies)
Transformamos las columnas categóricas (`cyl`, `gear`, `carb`) en variables dummies y calculamos las métricas.

In [4]:
# Crear dummies dropeando la primera categoría para evitar la trampa de colinealidad
df_dummies = pd.get_dummies(df_base, columns=['cyl', 'gear', 'carb'], drop_first=True)

X_21 = df_dummies.drop(columns=['mpg'])
y_21 = df_dummies['mpg']

# 1. Modelo Completo
modelo_21 = LinearRegression()
modelo_21.fit(X_21, y_21)
print(f"R2 Modelo Completo: {modelo_21.score(X_21, y_21):.4f}")

print("\nBetas (Coeficientes con Dummies):")
for feature, coef in zip(X_21.columns, modelo_21.coef_):
    print(f" - {feature}: {coef:.4f}")

# 2. Train-Test Split (40% train)
X_train_21, X_test_21, y_train_21, y_test_21 = train_test_split(X_21, y_21, train_size=0.4, random_state=42)
modelo_21_split = LinearRegression()
modelo_21_split.fit(X_train_21, y_train_21)
print(f"\nR2 Entrenamiento: {modelo_21_split.score(X_train_21, y_train_21):.4f}")
print(f"R2 Prueba: {modelo_21_split.score(X_test_21, y_test_21):.4f}")

R2 Modelo Completo: 0.8931

Betas (Coeficientes con Dummies):
 - disp: 0.0355
 - hp: -0.0705
 - drat: 1.1828
 - wt: -4.5298
 - qsec: 0.3678
 - vs: 1.9309
 - am: 1.2121
 - cyl_6: -2.6487
 - cyl_8: -0.3362
 - gear_4: 1.1144
 - gear_5: 2.5284
 - carb_2: -0.9794
 - carb_3: 2.9996
 - carb_4: 1.0914
 - carb_6: 4.4776
 - carb_8: 7.2504

R2 Entrenamiento: 1.0000
R2 Prueba: -1.3253


**Interpretación 2.1:**
* Al usar variables dummies, la primera categoría funciona como nuestro punto de referencia o "base" (ej. autos de 4 cilindros, 3 engranajes).
* **Betas de Cilindros (`cyl_6`: -2.64, `cyl_8`: -0.33):** Tener 6 u 8 cilindros disminuye el rendimiento (mpg) en comparación con tener 4 cilindros. Curiosamente, en este modelo lineal multivariable, pasar a 6 cilindros tiene un impacto negativo más fuerte que pasar a 8, ya que otras variables (como el peso o el motor) están absorbiendo la varianza.
* **Betas de Engranajes (`gear_4`: +1.11, `gear_5`: +2.52):** Un auto con 4 o 5 engranajes aumenta notablemente el `mpg` respecto a un auto de solo 3 engranajes.
* **Train/Test:** El sobreajuste en el split de entrenamiento (1.0000) es absoluto, ya que al crear dummies aumentamos la cantidad de columnas (dimensiones), permitiendo al modelo memorizar los limitados datos del 40% de la muestra.

## Actividad 2.2: Regresión con 'qsec' (Usando Variables Dummies)
Igual que el paso anterior, pero predecimos `qsec`.

In [5]:
X_22 = df_dummies.drop(columns=['qsec'])
y_22 = df_dummies['qsec']

modelo_22 = LinearRegression()
modelo_22.fit(X_22, y_22)
print(f"R2 Modelo Completo: {modelo_22.score(X_22, y_22):.4f}")

X_train_22, X_test_22, y_train_22, y_test_22 = train_test_split(X_22, y_22, train_size=0.4, random_state=42)
modelo_22_split = LinearRegression()
modelo_22_split.fit(X_train_22, y_train_22)
print(f"R2 Entrenamiento: {modelo_22_split.score(X_train_22, y_train_22):.4f}")
print(f"R2 Prueba: {modelo_22_split.score(X_test_22, y_test_22):.4f}")

R2 Modelo Completo: 0.9083
R2 Entrenamiento: 1.0000
R2 Prueba: -0.0600


## Actividad 3: Comparación de R2

**3.1 Comparación para 'mpg' (1.1 vs 2.1):**
* **Modelo 1.1 (Sin Dummies):** R2 Completo: 0.8690 | R2 Train: 0.9982 | R2 Test: -7.1071
* **Modelo 2.1 (Con Dummies):** R2 Completo: 0.8931 | R2 Train: 1.0000 | R2 Test: -1.3253
* **Conclusión:** Añadir variables dummies mejora la capacidad del modelo para explicar el total de los datos (R2 Completo sube a 89.3%), ya que permite que los cambios de categoría (ej. pasar de 4 a 6 cilindros) tengan impactos independientes. Al evaluar el split, observamos que las dummies llevaron al modelo a una "memorización" perfecta (R2 Train de 1.0) por el exceso de columnas frente a tan pocos datos, pero curiosamente el error en datos nuevos (Test) fue menos destructivo (-1.32 vs -7.10).

**3.2 Comparación para 'qsec' (1.2 vs 2.2):**
* **Modelo 1.2 (Sin Dummies):** R2 Completo: 0.8747 | R2 Train: 0.9989 | R2 Test: -1.0013
* **Modelo 2.2 (Con Dummies):** R2 Completo: 0.9083 | R2 Train: 1.0000 | R2 Test: -0.0600
* **Conclusión:** El comportamiento es idéntico. Tratar características ordinales mediante dummies elevó la precisión descriptiva sobre el total de datos (superando el 90% de la varianza explicada). Al evaluar el modelo en el set de prueba, las dummies amortiguaron enormemente el margen de error del sobreajuste (-0.06 frente a -1.00), dejándolo muy cerca de hacer predicciones neutrales frente a predecir puro ruido. El sobreajuste es un subproducto inevitable de tener un `train_size` del 40% (solo unos ~12 autos) para ajustar tantas variables.